<a href="https://colab.research.google.com/github/yash-clear/Web-Scrapping-and-Machine-Learning/blob/main/WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Problem 1

##Write a web scraping script which takes an input of any film actor and gives the output of filmography of that actor in descending order.



In [76]:
!pip install requests --upgrade  --quiet 
import requests

In [77]:
Filmography_url = 'https://en.wikipedia.org/wiki/Category:Indian_filmographies'
response = requests.get(Filmography_url)

In [78]:
response.status_code    

200

In [79]:
page_contents = response.text


In [80]:
with open('webpage.html','w') as f: 
    f.write(page_contents)

In [81]:
!pip install beautifulsoup4 --upgrade  --quiet 
from bs4 import BeautifulSoup
doc = BeautifulSoup(page_contents, 'html.parser')  #Now 'doc' contains entire html in parsed format

In [82]:
title = doc.find('title')
title

<title>Category:Indian filmographies - Wikipedia</title>

In [83]:
# This is to get the name of actors/actress.

a_tags = doc.find_all('a',{'class': None})
len(a_tags)

260

In [84]:
titles_a_tag = []
for tag in a_tags:
    if 'filmography' in tag.text:
        titles_a_tag.append(tag['title'].replace('filmography',''))
titles_a_tag[:5]

['John Abraham ',
 'Kajal Aggarwal ',
 'Samantha Ruth Prabhu ',
 'Ali ',
 'Zeenat Aman ']

In [85]:
# Now we need to fetch the URL's of all the filmographies
# I have taken all the filmographies url which has "filmography" in all of them.
link_a_tag = []
for tag in a_tags:
    if 'filmography' in tag.text:
        link_a_tag.append(tag['href'])
link_a_tag[:10]

['/wiki/John_Abraham_filmography',
 '/wiki/Kajal_Aggarwal_filmography',
 '/wiki/Samantha_Ruth_Prabhu_filmography',
 '/wiki/Ali_filmography',
 '/wiki/Zeenat_Aman_filmography',
 '/wiki/Ambareesh_filmography',
 '/wiki/Dev_Anand_filmography',
 '/wiki/Anil_Panachooran_filmography',
 '/wiki/Ramesh_Aravind_filmography',
 '/wiki/Allu_Arjun_filmography']

In [86]:
# To get the full link we need to give the "Base URL" which is given below
Filmography0_url = "https://en.wikipedia.org/"+ link_a_tag[0]
Filmography0_url # This is the first URl.

'https://en.wikipedia.org//wiki/John_Abraham_filmography'

In [ ]:
# WE need to loop over to get all the URl's
Filmography_urls = []

for i in range(len(link_a_tag)):
    
    Filmography_urls.append("https://en.wikipedia.org/"+ link_a_tag[i])

In [88]:
!pip install pandas --upgrade --quiet    #Installing Pandas Library
import pandas as pd

In [89]:
Filmography_dict = {
    'Actor/Actress_Name' : titles_a_tag,
    'Url_Of_Profiles': Filmography_urls,
}

In [90]:
Filmography_df = pd.DataFrame(Filmography_dict)  #Here we convert the dictionary into a Pandas DataFrame

In [91]:
Filmography_df

,Actor/Actress_Name,Url_Of_Profiles
0,John Abraham,https://en.wikipedia.org//wiki/John_Abraham_fi...
1,Kajal Aggarwal,https://en.wikipedia.org//wiki/Kajal_Aggarwal_...
2,Samantha Ruth Prabhu,https://en.wikipedia.org//wiki/Samantha_Ruth_P...
3,Ali,https://en.wikipedia.org//wiki/Ali_filmography
4,Zeenat Aman,https://en.wikipedia.org//wiki/Zeenat_Aman_fil...
...,...,...
182,Savitri,https://en.wikipedia.org//wiki/Savitri_filmogr...
183,Seeman,https://en.wikipedia.org//wiki/Seeman_filmography
184,Aparna Sen,https://en.wikipedia.org//wiki/Aparna_Sen_film...
185,Jisshu Sengupta,https://en.wikipedia.org//wiki/Jisshu_Sengupta...


In [110]:
name = input()

Samantha Ruth Prabhu 


In [118]:
index = 0
for i in Filmography_df['Actor/Actress_Name']:
  if i == name:
    index
    break
  index+=1

In [119]:
index 

2

In [120]:
urls = [Filmography_df['Url_Of_Profiles'][index]]


In [121]:
urls

['https://en.wikipedia.org//wiki/Samantha_Ruth_Prabhu_filmography']

In [122]:
# get_docs fuction will reterive all the tables which has class "Wikitable"
# and also get_docs function will get all the "h1" tags and replace filmography with empty space so that we will get all the Actor/Actress name.
def get_docs(films_urls):
    doc_list = []
    title_list = []
    for url in films_urls:
        # Fetch and parse url's using Bs4
        response = requests.get(url)
        response.status_code
        if response.status_code == 200 : # check the status_code 
            topic_doc = BeautifulSoup(response.text,'html.parser')
            title_tags = topic_doc.find('h1').text.replace(' filmography','')
            body_tags = topic_doc.find_all('table',{'class' : 'wikitable'})
            doc_list.append(body_tags)
            title_list.append(title_tags)
        else :
            print('Failed to load page {}'.format(url)) # if status_code is not equal to 200
    return doc_list,title_list

In [123]:
Table_urls, title_tags = get_docs(urls) 

In [124]:
title_tags

['Samantha Ruth Prabhu']

In [125]:
csv = 'my_file.csv'
def DataFrames(Tables):
    dfs = []
    for i in Tables:
        df=pd.read_html(str(i))
        df[0].to_csv(csv)
        dfs.append(df)
    return df
    

In [127]:
# here we have the names of few actor/actress which we have taken in our data.
# we are adding the actor/actress column in the dataframe. 
dfs = DataFrames(Table_urls)
for df, title in zip(dfs, title_tags) :
    df['Actor/Actress'] = title
dfs

[    0                                              1         Actor/Actress
 0 NaN  Denotes films that have not yet been released  Samantha Ruth Prabhu,
     Year                               Title  \
 0   2010                    Ye Maaya Chesave   
 1   2010            Vinnaithaandi Varuvaayaa   
 2   2010                      Baana Kaathadi   
 3   2010                     Moscowin Kavery   
 4   2010                         Brindavanam   
 5   2011                    Nadunisi Naaygal   
 6   2011                             Dookudu   
 7   2012                     Ekk Deewana Tha   
 8   2012                                Eega   
 9   2012                             Naan Ee   
 10  2012           Neethaane En Ponvasantham   
 11  2012           Yeto Vellipoyindhi Manasu   
 12  2013  Seethamma Vakitlo Sirimalle Chettu   
 13  2013                          Jabardasth   
 14  2013       Theeya Velai Seiyyanum Kumaru   
 15  2013                  Attarintiki Daredi   
 16  2013     

In [128]:
Table_contents = pd.concat(dfs) # we have concatinate all dataframes using pd.concat()
# Table_contents

In [129]:
df = Table_contents[['Title']]

In [130]:
df.head()

,Title
0,NaN
0,Ye Maaya Chesave
1,Vinnaithaandi Varuvaayaa
2,Baana Kaathadi
3,Moscowin Kavery
